In [61]:
import networkx as nx
import numpy as np
import operator
from collections import defaultdict
from itertools import permutations, product
from functools import partial
import scipy.sparse as sps
import scipy.linalg as la
from scipy.optimize import minimize, linear_sum_assignment, LinearConstraint, Bounds
import sys
import time
import importlib

sys.path.insert(1, '/home/me/mGH_UB')
importlib.reload(sys.modules['ub'])
from ub import *
importlib.reload(sys.modules['tools'])
from tools import *
sys.path.insert(1, '/home/me/persim-fork/persim')
from gromov_hausdorff import estimate, find_lb, find_ub, find_ub_of_min_distortion


np.set_printoptions(suppress=True)

In [67]:
rnd = np.random.RandomState(0)
n = 100
_, X = gen_random_graph(n, .05)
f = np.random.permutation(len(X))
Y = X[np.ix_(f, f)]

In [68]:
rnd = np.random.RandomState(0)

find_min_dis(X, Y, n_restarts=10, c_seq=[1.1, 2], rnd=rnd, verbose=1)

iter 1: dis(P)=3.060, dis(Proj P)=6.0, u(P)=19761.41, (to) F, γ=0.21075 
iter 2: dis(P)=3.354, dis(Proj P)=5.0, u(P)=19754.88, (to) F, γ=0.12707 
iter 3: dis(P)=3.357, dis(Proj P)=5.0, u(P)=19750.54, (to) F, γ=0.31597 
iter 4: dis(P)=3.042, dis(Proj P)=4.0, u(P)=19740.45, (to) F, γ=0.32364 
iter 5: dis(P)=3.354, dis(Proj P)=5.0, u(P)=19729.82, (to) F, γ=0.56976 
iter 6: dis(P)=3.723, dis(Proj P)=4.0, u(P)=19712.67, (to) F, γ=1.00000 
iter 7: dis(P)=4.000, dis(Proj P)=4.0, u(P)=19678.05, (to) F, γ=0.34258 
iter 8: dis(P)=3.657, dis(Proj P)=4.0, u(P)=19671.50, (to) F, γ=1.00000 
iter 9: dis(P)=4.000, dis(Proj P)=4.0, u(P)=19656.79, (to) F, γ=1.00000 
iter 10: dis(P)=4.000, dis(Proj P)=4.0, u(P)=19645.04, (to) F, γ=1.00000 
----------FINAL dis(P)=3.000, dis(Proj P)=4.0, u(P)=19612.91
iter 1: dis(P)=3.000, dis(Proj P)=3.0, u(P)=20264.50, (to) F, γ=1.00000 
----------STOP: (to) F, γ=0.50000, ‖∇u(P)‖²=58654.47, ‖γD‖²=0.00 
----------FINAL dis(P)=0.000, dis(Proj P)=0.0, u(P)=19602.00
iter 1: 

iter 2: dis(P)=3.353, dis(Proj P)=5.0, u(P)=19754.69, (to) F, γ=0.14838 
iter 3: dis(P)=3.339, dis(Proj P)=5.0, u(P)=19749.56, (to) F, γ=0.22589 
iter 4: dis(P)=3.069, dis(Proj P)=4.0, u(P)=19742.26, (to) F, γ=0.45414 
iter 5: dis(P)=3.405, dis(Proj P)=5.0, u(P)=19728.00, (to) F, γ=0.40527 
iter 6: dis(P)=3.644, dis(Proj P)=4.0, u(P)=19714.93, (to) F, γ=1.00000 
iter 7: dis(P)=4.000, dis(Proj P)=4.0, u(P)=19687.13, (to) F, γ=0.57657 
iter 8: dis(P)=3.000, dis(Proj P)=4.0, u(P)=19671.10, (to) F, γ=1.00000 
iter 9: dis(P)=3.000, dis(Proj P)=3.0, u(P)=19643.92, (to) F, γ=1.00000 
----------STOP: (to) F, γ=0.50000, ‖∇u(P)‖²=39488.17, ‖γD‖²=0.00 
----------FINAL dis(P)=0.000, dis(Proj P)=0.0, u(P)=19602.00
----------STOP: (to) F, γ=0.50000, ‖∇u(P)‖²=58654.47, ‖γD‖²=0.00 
----------FINAL dis(P)=0.000, dis(Proj P)=0.0, u(P)=19602.00
iter 1: dis(P)=3.147, dis(Proj P)=5.0, u(P)=19761.17, (to) F, γ=0.20966 
iter 2: dis(P)=3.284, dis(Proj P)=5.0, u(P)=19754.57, (to) F, γ=0.13068 
iter 3: dis(P)=3

(0.0,
 array([44, 47, 13, 72, 21, 25, 79, 81,  6, 91,  1, 88, 27, 86, 83, 71, 26,
        82, 94, 31, 35, 20, 12,  2, 60, 37, 68, 66, 45, 55, 30,  3, 22, 18,
        34, 76, 36, 63, 29, 49, 98, 43, 42,  5, 87, 24, 52,  7, 46, 57, 96,
        85, 80, 15, 62, 39, 41, 69, 61, 48, 97, 78,  4,  9,  8, 92, 70, 10,
        65, 90, 23, 95, 19, 38, 40, 58, 11, 17, 59, 74, 75, 32, 93, 28, 50,
        56, 16, 64, 89, 51, 14, 73,  0, 33, 53, 67, 77, 84, 54]))

In [71]:
min_dis = np.inf
for _ in range(1000):
    min_dis = min(min_dis, find_ub_of_min_distortion(X, Y))

min_dis

2.0

In [2]:
def make_functions(DX, DY, ts=[None, .01], cs=[1.1]):
    obj_types = ['XP-PY', 'sq+', 'exp', 'exp_2', 'sq'][1:4]
    mx_types = ['D', 'K-inv', 'K-exp', 'K-exp_t']
    gen_mapping_ = partial(gen_mapping, n=len(DX), m=len(DY))
    gen_P_ = partial(gen_stoch_mx, len(DX), len(DY))
    p2P_ = partial(p2P, m=len(DY))
    dis_ = partial(dis, DX=DX, DY=DY)
    convs = dict()
    fs = dict()
    grads = dict()
    fws = dict()
    for t in ts:
        X, Y = (D2K(DX, t=t), D2K(DY, t=t)) if t else (DX, DY)
        for c in cs:
            key = t, c
            convs[key] = conv(c_exp_hess(c, X, Y))
            fs[key], grads[key], alpha_jac = def_functions_for_frank_wolfe(X, Y, 'c_exp', c=c)
            fws[key] = partial(frank_wolfe, len(DX), len(DY), fs[key], grads[key], alpha_jac, dis_)
        
    return gen_P_, p2P_, gen_mapping_, dis_, grads, fs, fws, convs

In [14]:
max_iter = 10
# An example where optimal map X → Y is [0, 1, 3, 3] or [1, 0, 3, 3] (non-injective):
DX = np.array([[0., 2, 10, 10], [2, 0, 10, 10], [10, 10, 0, 1], [10, 10, 1, 0]])
DY = np.array([[0, 1, .5, 10], [1, 0, .5, 10], [.5, .5, 0, 10], [10, 10, 10, 0]])
ts = [None] + list(.025 * 2**np.arange(8))
cs = list(1 + .01*2**np.arange(10))
gen_P_, p2P_, gen_mapping_, dis_, grads, fs, fws, convs = make_functions(DX, DY, ts=ts, cs=cs)


In [19]:
grads[None, 1.32](p2P_([3,3,0,1]))

array([[107.63658692, 107.63658692, 107.09424287,  16.63264206],
       [107.63658692, 107.63658692, 107.09424287,  16.63264206],
       [ 16.        ,  16.31030303,  16.07720324, 121.22959258],
       [ 16.31030303,  16.        ,  16.07720324, 121.22959258]])

In [98]:
# Find Q (descent direction) from each mapping (vertex) Phi:
for key in [(None, 1.01)]:
    print(key)
    for Phi in gen_mapping_(is_inj=False):
        print(f'{tuple(P2p(Phi))}→{tuple(find_descent_direction(grads[key](Phi), return_matrix=False))}')

(None, 1.01)
(0, 0, 0, 0)→(3, 3, 1, 1)
(0, 0, 0, 1)→(3, 3, 1, 1)
(0, 0, 0, 2)→(3, 3, 1, 1)
(0, 0, 0, 3)→(1, 1, 3, 3)
(0, 0, 1, 0)→(3, 3, 1, 1)
(0, 0, 1, 1)→(3, 3, 1, 1)
(0, 0, 1, 2)→(3, 3, 1, 1)
(0, 0, 1, 3)→(0, 0, 3, 3)
(0, 0, 2, 0)→(3, 3, 1, 1)
(0, 0, 2, 1)→(3, 3, 1, 1)
(0, 0, 2, 2)→(3, 3, 1, 1)
(0, 0, 2, 3)→(1, 1, 3, 3)
(0, 0, 3, 0)→(1, 1, 3, 3)
(0, 0, 3, 1)→(0, 0, 3, 3)
(0, 0, 3, 2)→(1, 1, 3, 3)
(0, 0, 3, 3)→(1, 1, 3, 3)
(0, 1, 0, 0)→(3, 3, 2, 2)
(0, 1, 0, 1)→(3, 3, 2, 2)
(0, 1, 0, 2)→(3, 3, 3, 2)
(0, 1, 0, 3)→(1, 1, 3, 3)
(0, 1, 1, 0)→(3, 3, 2, 2)
(0, 1, 1, 1)→(3, 3, 2, 2)
(0, 1, 1, 2)→(3, 3, 3, 2)
(0, 1, 1, 3)→(0, 0, 3, 3)
(0, 1, 2, 0)→(3, 3, 2, 3)
(0, 1, 2, 1)→(3, 3, 2, 3)
(0, 1, 2, 2)→(3, 3, 3, 3)
(0, 1, 2, 3)→(0, 1, 3, 3)
(0, 1, 3, 0)→(1, 1, 3, 3)
(0, 1, 3, 1)→(0, 0, 3, 3)
(0, 1, 3, 2)→(0, 1, 3, 3)
(0, 1, 3, 3)→(0, 1, 3, 3)
(0, 2, 0, 0)→(3, 3, 1, 1)
(0, 2, 0, 1)→(3, 3, 1, 1)
(0, 2, 0, 2)→(3, 3, 1, 1)
(0, 2, 0, 3)→(1, 1, 3, 3)
(0, 2, 1, 0)→(3, 3, 1, 1)
(0, 2, 1, 1)→(3, 3, 1, 1)

In [135]:
# Mappings' distortions and optimal mappings.
Phi_distortions = {tuple(P2p(Phi)): dis_(Phi) for Phi in gen_mapping_()}
Phi_distortions_unique = np.unique(list(Phi_distortions.values()), return_counts=True)
print("mappings' distortions w/ counts:", Phi_distortions_unique)
min_dis = Phi_distortions_unique[0][0]
opt_Phis = {x: distortion for x, distortion in Phi_distortions.items() if distortion == min_dis}
print('optimal mappings:', opt_Phis)

mappings' distortions w/ counts: (array([ 1. ,  1.5,  9. ,  9.5, 10. ]), array([  6,  12,  10,  56, 172]))
optimal mappings: {(0, 1, 3, 3): 1.0, (0, 2, 3, 3): 1.0, (1, 0, 3, 3): 1.0, (1, 2, 3, 3): 1.0, (2, 0, 3, 3): 1.0, (2, 1, 3, 3): 1.0}


In [122]:
# Objective function values and those P that minimize it.
for key in fs:
    all_f_vals = {tuple(P2p(Phi)): fs[key](Phi) for Phi in gen_mapping_()}
    all_f_vals_unique = np.unique(list(all_f_vals.values()), return_counts=True)
    print(key)
    print([x for x, f_val in all_f_vals.items() if f_val == all_f_vals_unique[0][0]])
    #print(all_obj_vals_unique)

(None, 1.01)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 1.02)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 1.04)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 1.08)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 1.16)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 1.32)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 1.6400000000000001)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 2.2800000000000002)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 3.56)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(None, 6.12)
[(0, 1, 3, 3), (1, 0, 3, 3)]
(0.025, 1.01)
[(0, 2, 3, 1), (0, 3, 2, 1), (1, 2, 3, 0), (1, 3, 2, 0), (2, 0, 1, 3), (2, 1, 0, 3), (3, 0, 1, 2), (3, 1, 0, 2)]
(0.025, 1.02)
[(0, 2, 1, 3), (0, 2, 3, 1), (0, 3, 1, 2), (0, 3, 2, 1), (1, 2, 0, 3), (1, 2, 3, 0), (1, 3, 0, 2), (1, 3, 2, 0), (2, 0, 1, 3), (2, 0, 3, 1), (2, 1, 0, 3), (2, 1, 3, 0), (3, 0, 1, 2), (3, 0, 2, 1), (3, 1, 0, 2), (3, 1, 2, 0)]
(0.025, 1.04)
[(0, 2, 1, 3), (0, 3, 1, 2), (1, 2, 0, 3), (1, 3, 0, 2), (2, 0, 3, 1), (2, 1, 3, 0), (3, 0, 2, 1), (3, 1, 2, 0)]
(0.025, 1.08)
[(0, 2, 1, 3), (0, 2, 3, 1), (0, 3, 1, 2), (0

(0.4, 1.02)
[(0, 2, 3, 1), (0, 3, 2, 1), (1, 2, 3, 0), (1, 3, 2, 0), (2, 0, 1, 3), (2, 1, 0, 3), (3, 0, 1, 2), (3, 1, 0, 2)]
(0.4, 1.04)
[(0, 2, 1, 3), (0, 2, 3, 1), (0, 3, 1, 2), (0, 3, 2, 1), (1, 2, 0, 3), (1, 2, 3, 0), (1, 3, 0, 2), (1, 3, 2, 0), (2, 0, 1, 3), (2, 0, 3, 1), (2, 1, 0, 3), (2, 1, 3, 0), (3, 0, 1, 2), (3, 0, 2, 1), (3, 1, 0, 2), (3, 1, 2, 0)]
(0.4, 1.08)
[(0, 2, 1, 3), (0, 2, 3, 1), (0, 3, 1, 2), (0, 3, 2, 1), (1, 2, 0, 3), (1, 2, 3, 0), (1, 3, 0, 2), (1, 3, 2, 0), (2, 0, 1, 3), (2, 0, 3, 1), (2, 1, 0, 3), (2, 1, 3, 0), (3, 0, 1, 2), (3, 0, 2, 1), (3, 1, 0, 2), (3, 1, 2, 0)]
(0.4, 1.16)
[(0, 2, 1, 3), (0, 2, 3, 1), (0, 3, 1, 2), (0, 3, 2, 1), (1, 2, 0, 3), (1, 2, 3, 0), (1, 3, 0, 2), (1, 3, 2, 0), (2, 0, 1, 3), (2, 0, 3, 1), (2, 1, 0, 3), (2, 1, 3, 0), (3, 0, 1, 2), (3, 0, 2, 1), (3, 1, 0, 2), (3, 1, 2, 0)]
(0.4, 1.32)
[(0, 2, 3, 1), (1, 2, 3, 0), (2, 0, 1, 3), (2, 0, 3, 1), (2, 1, 0, 3), (2, 1, 3, 0)]
(0.4, 1.6400000000000001)
[(0, 2, 1, 3), (0, 2, 3, 1), (0, 3, 1, 2)

In [7]:
P_star, P = p2P_([0,1,3,3]), p2P_([3,3,0,1])
grads[None, 1.08](P_star), grads[None, 1.08](P)
c = 2
a = 0
f, grad, _ = def_functions_for_frank_wolfe(DX, DY, 'c_exp', c=c)
grad(a*P_star + (1-a)*P), f(P_star), f(P)
grad(p2P_([3, 3, 0, 0]))
#fws[None, 1.08](verbose=True)

array([[6656.01367188, 4608.01757812, 5456.32466564,   20.5       ],
       [6656.01367188, 4608.01757812, 5456.32466564,   20.5       ],
       [  17.        ,   17.        ,   16.48528137, 7168.00976562],
       [  17.        ,   17.        ,   16.48528137, 7168.00976562]])

In [127]:
# for key, fw in fws.items():
for t, c in product([t for t in ts if t], cs):
    fw_seq = [fws[t, c]] + [fws[None, c_] for c_ in cs] # K + D for each c
    fw_seq = [fws[t, c_] for c_ in cs] + [fws[None, c]] # K for each c + D
    if dis_(project_P(frank_wolfe_sequence(fw_seq))) == 5:
        print(t, c)

In [145]:
for t in [t for t in ts if t]:
    KX, KY = D2K(DX, t=t), D2K(DY, t=t)
    print(f' -- t={t}')
    for c in [1.01, 1.1, 1.5, 2, 3, 4]:
        fw = partial(frank_wolfe, len(DX), len(DY), *def_functions_for_frank_wolfe(KX, KY, 'c_exp', c=c), dis_)
        success = []
        for _ in range(5000):
            success.append(dis_(project_P(fw(P0=gen_P_()))) == 1)
        print(f'c={c}: {np.mean(success):.4%}')

 -- t=0.1
c=2: 0.9200%


In [13]:
# FW solutions & their distortions (from all possible mappings as initial guesses).
Pstar_distortions = dict()
Pstars = dict()
n_f = len(DY)**len(DX)
for key in fw:
    print(key)
    Pstar_distortions[key] = dict()
    Pstars[key] = dict()
    for P0 in generate_f_(is_inj=False):
        P0_tuple = tuple(P2p(P0))
        P = fw[key](is_inj=False, P0=P0, verbose=False, max_iter=100)
        Pstar_distortions[key][P0_tuple] = dis_(project_P(P))
        Pstars[key][P0_tuple] = tuple(P2p(project_P(P)))
        if Pstars[key][P0_tuple] != P0_tuple:
            print(f'{P0_tuple} → {Pstars[key][P0_tuple]}')

('sq+', 'D')
(0, 0, 0, 0) → (3, 3, 3, 3)
(0, 0, 0, 1) → (3, 3, 1, 0)
(0, 0, 0, 2) → (3, 3, 0, 1)
(0, 0, 0, 3) → (1, 1, 3, 3)
(0, 0, 1, 0) → (3, 3, 0, 1)
(0, 0, 1, 1) → (3, 3, 1, 1)
(0, 0, 1, 2) → (3, 3, 1, 0)
(0, 0, 1, 3) → (0, 0, 3, 3)
(0, 0, 2, 0) → (3, 3, 1, 0)
(0, 0, 2, 1) → (3, 3, 0, 1)
(0, 0, 2, 2) → (3, 3, 1, 1)
(0, 0, 2, 3) → (1, 1, 3, 3)
(0, 0, 3, 0) → (1, 1, 3, 3)
(0, 0, 3, 1) → (0, 0, 3, 3)
(0, 0, 3, 2) → (1, 1, 3, 3)
(0, 0, 3, 3) → (3, 3, 1, 0)
(0, 1, 0, 0) → (3, 3, 0, 0)
(0, 1, 0, 1) → (0, 1, 3, 3)
(0, 1, 0, 2) → (3, 3, 0, 1)
(0, 1, 0, 3) → (0, 1, 3, 3)
(0, 1, 1, 0) → (0, 1, 3, 3)
(0, 1, 1, 1) → (3, 3, 1, 1)
(0, 1, 1, 2) → (3, 3, 1, 0)
(0, 1, 1, 3) → (0, 1, 3, 3)
(0, 1, 2, 0) → (3, 3, 1, 0)
(0, 1, 2, 1) → (3, 3, 0, 1)
(0, 1, 2, 2) → (3, 3, 1, 0)
(0, 1, 2, 3) → (0, 1, 3, 3)
(0, 1, 3, 0) → (0, 1, 3, 3)
(0, 1, 3, 1) → (0, 1, 3, 3)
(0, 1, 3, 2) → (0, 1, 3, 3)
(0, 2, 0, 0) → (3, 3, 0, 0)
(0, 2, 0, 1) → (3, 3, 1, 0)
(0, 2, 0, 2) → (1, 0, 3, 3)
(0, 2, 0, 3) → (0, 1, 3, 3)
(0, 2, 

(0, 3, 0, 1) → (0, 1, 2, 1)
(0, 3, 0, 2) → (0, 1, 1, 2)
(0, 3, 0, 3) → (2, 1, 0, 2)
(0, 3, 1, 0) → (0, 1, 1, 2)
(0, 3, 1, 1) → (0, 2, 1, 1)
(0, 3, 1, 2) → (0, 1, 1, 2)
(0, 3, 1, 3) → (0, 1, 1, 2)
(0, 3, 2, 0) → (0, 1, 2, 1)
(0, 3, 2, 1) → (0, 1, 2, 1)
(0, 3, 2, 2) → (0, 1, 2, 2)
(0, 3, 2, 3) → (0, 2, 2, 1)
(0, 3, 3, 0) → (2, 1, 2, 0)
(0, 3, 3, 1) → (0, 1, 2, 1)
(0, 3, 3, 2) → (0, 2, 1, 2)
(0, 3, 3, 3) → (0, 2, 3, 3)
(1, 0, 0, 0) → (3, 3, 3, 3)
(1, 0, 0, 1) → (2, 2, 1, 0)
(1, 0, 0, 3) → (1, 0, 0, 2)
(1, 0, 1, 0) → (2, 2, 0, 1)
(1, 0, 1, 1) → (3, 3, 3, 3)
(1, 0, 1, 3) → (1, 0, 1, 2)
(1, 0, 2, 2) → (1, 2, 0, 0)
(1, 0, 2, 3) → (1, 2, 2, 0)
(1, 0, 3, 0) → (1, 0, 2, 0)
(1, 0, 3, 1) → (1, 0, 2, 1)
(1, 0, 3, 2) → (1, 2, 0, 2)
(1, 1, 0, 1) → (2, 2, 1, 0)
(1, 1, 0, 3) → (1, 1, 0, 2)
(1, 1, 1, 0) → (2, 2, 0, 1)
(1, 1, 1, 1) → (3, 3, 3, 3)
(1, 1, 1, 2) → (0, 0, 1, 2)
(1, 1, 1, 3) → (0, 0, 1, 2)
(1, 1, 2, 1) → (0, 0, 2, 1)
(1, 1, 2, 3) → (1, 1, 2, 0)
(1, 1, 3, 0) → (1, 1, 2, 0)
(1, 1, 3, 1) → (0, 0

(2, 2, 1, 2) → (3, 3, 2, 2)
(2, 2, 1, 3) → (1, 0, 3, 3)
(2, 2, 2, 0) → (3, 3, 1, 0)
(2, 2, 2, 1) → (3, 3, 2, 0)
(2, 2, 2, 2) → (3, 3, 1, 0)
(2, 2, 2, 3) → (1, 0, 3, 3)
(2, 2, 3, 0) → (0, 0, 3, 3)
(2, 2, 3, 1) → (0, 1, 3, 3)
(2, 2, 3, 2) → (1, 0, 3, 3)
(2, 2, 3, 3) → (0, 0, 3, 3)
(2, 3, 0, 0) → (3, 3, 2, 2)
(2, 3, 0, 1) → (3, 3, 0, 1)
(2, 3, 0, 2) → (3, 3, 0, 1)
(2, 3, 0, 3) → (1, 3, 0, 3)
(2, 3, 1, 0) → (3, 3, 1, 0)
(2, 3, 1, 1) → (3, 3, 2, 0)
(2, 3, 1, 2) → (3, 3, 1, 0)
(2, 3, 1, 3) → (0, 3, 1, 3)
(2, 3, 2, 0) → (3, 3, 1, 0)
(2, 3, 2, 1) → (3, 3, 0, 1)
(2, 3, 2, 2) → (3, 3, 1, 0)
(2, 3, 2, 3) → (1, 3, 1, 3)
(2, 3, 3, 0) → (1, 3, 3, 0)
(2, 3, 3, 1) → (0, 3, 3, 1)
(2, 3, 3, 2) → (0, 3, 3, 0)
(2, 3, 3, 3) → (0, 1, 3, 3)
(3, 0, 0, 0) → (3, 3, 2, 0)
(3, 0, 0, 1) → (3, 3, 2, 2)
(3, 0, 0, 2) → (3, 3, 2, 1)
(3, 0, 0, 3) → (3, 3, 2, 2)
(3, 0, 1, 0) → (3, 3, 0, 1)
(3, 0, 1, 2) → (3, 3, 1, 0)
(3, 0, 2, 0) → (3, 3, 2, 2)
(3, 0, 2, 1) → (3, 3, 2, 2)
(3, 0, 2, 2) → (3, 3, 0, 1)
(3, 0, 2, 3) → (3, 0

(3, 2, 2, 2) → (3, 3, 3, 3)
(3, 2, 2, 3) → (3, 3, 3, 3)
(3, 2, 3, 0) → (3, 3, 3, 3)
(3, 2, 3, 1) → (3, 3, 3, 3)
(3, 2, 3, 2) → (3, 3, 3, 3)
(3, 2, 3, 3) → (3, 3, 3, 3)
(3, 3, 0, 0) → (3, 3, 3, 3)
(3, 3, 0, 1) → (3, 3, 3, 3)
(3, 3, 0, 2) → (3, 3, 3, 3)
(3, 3, 0, 3) → (3, 3, 3, 3)
(3, 3, 1, 0) → (3, 3, 3, 3)
(3, 3, 1, 1) → (3, 3, 3, 3)
(3, 3, 1, 2) → (3, 3, 3, 3)
(3, 3, 1, 3) → (3, 3, 3, 3)
(3, 3, 2, 0) → (3, 3, 3, 3)
(3, 3, 2, 1) → (3, 3, 3, 3)
(3, 3, 2, 2) → (3, 3, 3, 3)
(3, 3, 2, 3) → (3, 3, 3, 3)
(3, 3, 3, 0) → (3, 3, 3, 3)
(3, 3, 3, 1) → (3, 3, 3, 3)
(3, 3, 3, 2) → (3, 3, 3, 3)
('exp_2', 'D')
(0, 0, 0, 0) → (3, 3, 2, 2)
(0, 0, 0, 1) → (1, 1, 3, 3)
(0, 0, 0, 2) → (3, 3, 2, 2)
(0, 0, 0, 3) → (1, 1, 3, 3)
(0, 0, 1, 0) → (1, 1, 3, 3)
(0, 0, 1, 1) → (3, 3, 2, 2)
(0, 0, 1, 2) → (3, 3, 2, 2)
(0, 0, 1, 3) → (1, 1, 3, 3)
(0, 0, 2, 0) → (3, 3, 2, 2)
(0, 0, 2, 1) → (3, 3, 2, 2)
(0, 0, 2, 2) → (3, 3, 2, 2)
(0, 0, 2, 3) → (1, 1, 3, 3)
(0, 0, 3, 0) → (1, 1, 3, 3)
(0, 0, 3, 1) → (0, 0, 3, 3)
(0, 0

(0, 1, 1, 1) → (3, 3, 3, 3)
(0, 1, 1, 2) → (3, 3, 3, 3)
(0, 1, 1, 3) → (3, 3, 3, 3)
(0, 1, 2, 0) → (3, 3, 3, 3)
(0, 1, 2, 1) → (3, 3, 3, 3)
(0, 1, 2, 2) → (3, 3, 3, 3)
(0, 1, 2, 3) → (3, 3, 3, 3)
(0, 1, 3, 0) → (3, 3, 3, 3)
(0, 1, 3, 1) → (3, 3, 3, 3)
(0, 1, 3, 2) → (3, 3, 3, 3)
(0, 1, 3, 3) → (3, 3, 3, 3)
(0, 2, 0, 0) → (3, 3, 3, 3)
(0, 2, 0, 1) → (3, 3, 3, 3)
(0, 2, 0, 2) → (3, 3, 3, 3)
(0, 2, 0, 3) → (3, 3, 3, 3)
(0, 2, 1, 0) → (3, 3, 3, 3)
(0, 2, 1, 1) → (3, 3, 3, 3)
(0, 2, 1, 2) → (3, 3, 3, 3)
(0, 2, 1, 3) → (3, 3, 3, 3)
(0, 2, 2, 0) → (3, 3, 3, 3)
(0, 2, 2, 1) → (3, 3, 3, 3)
(0, 2, 2, 2) → (3, 3, 3, 3)
(0, 2, 2, 3) → (3, 3, 3, 3)
(0, 2, 3, 0) → (3, 3, 3, 3)
(0, 2, 3, 1) → (3, 3, 3, 3)
(0, 2, 3, 2) → (3, 3, 3, 3)
(0, 2, 3, 3) → (3, 3, 3, 3)
(0, 3, 0, 0) → (3, 3, 3, 3)
(0, 3, 0, 1) → (3, 3, 3, 3)
(0, 3, 0, 2) → (3, 3, 3, 3)
(0, 3, 0, 3) → (3, 3, 3, 3)
(0, 3, 1, 0) → (3, 3, 3, 3)
(0, 3, 1, 1) → (3, 3, 3, 3)
(0, 3, 1, 2) → (3, 3, 3, 3)
(0, 3, 1, 3) → (3, 3, 3, 3)
(0, 3, 2, 0) → (3, 3

In [48]:
# Print each FW solution with the number of initial guesses (mappings) led to it
for key in fw:
    print(key)
    print([f'{d}: {count/n_f:.2%}' for d, count in zip(*np.unique(list(Pstar_distortions[key].values()), return_counts=True))])
    unique_Pstar_counts = defaultdict(int)
    for Pstar in Pstars[key].values():
        unique_Pstar_counts[Pstar] += 1
        #unique_Pstar_counts[tuple(tuple(np.round(row, 10)) for row in Pstar)] += 1
    sorted_Pstars = sorted(unique_Pstar_counts.items(), key=operator.itemgetter(1), reverse=True)
    n = 0
    for i, (_, count) in enumerate(sorted_Pstars):
        n += count
        if n > 0.95*n_f:
            break
        
    print('P*:', [f'{Pstar}: {count/n_f:.2%}' for Pstar, count in sorted_Pstars[:i+1]])

('sq+', 'D')


NameError: name 'Pstar_distortions' is not defined

In [49]:
# FW solutions & their distortions (from random initial guesses).
Pstar_rnd_distortions = dict()
Pstars_rnd = dict()
n_rnd = 100
is_inj = False
for key in fw:
    print(key)
    Pstar_rnd_distortions[key] = defaultdict(int)
    Pstars_rnd[key] = defaultdict(int)
    for _ in range(n_rnd):
        P0 = np.random.rand(len(DX), len(DY))
        if is_inj: # |X| ≤ |Y|
            assert len(DX) == len(DY), 'can only generate _square_ bi-stochastic matrices'
            row_sum, col_sum = 0, 0
            while not np.allclose(row_sum, 1) or not np.allclose(col_sum, 1):
                P0 /= P0.sum(axis=0)
                P0 /= P0.sum(axis=1)[:, None]
                row_sum, col_sum = P0.sum(axis=1), P0.sum(axis=0)
        else:
            P0 /= P0.sum(axis=1)[:, None]
        
        P = fw[key](is_inj=is_inj, P0=P0, verbose=False, max_iter=100)
        proj_P = p2P_(linear_sum_assignment(P)[1]) if is_inj else project_P(P)
        Pstar_rnd_distortions[key][dis_(proj_P)] += 1
        Pstars_rnd[key][tuple(P2p(proj_P))] += 1


('sq+', 'D')
('sq+', 'K')
('exp', 'D')
('exp', 'K')
('exp_2', 'D')
('exp_2', 'K')


In [5]:
AX, DX = gen_graph(100, 0.05)
pi_YX = np.random.permutation(len(DX))
DY = DX[np.ix_(pi_YX, pi_YX)]
dis_ = partial(dis, DX, DY)
fw = make_fw(DX, DY, c=2, dis_=dis_)

In [11]:
P_star, n_iter = fw(verbose=True)

iter 0: dis(P)=1528.255, dis(Proj P)=1548.0, f(P)=30622.52, proj P=[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], Q=[ 7 63 63 63 40 63 63 21  7 63 63  3 10 63 63  7  7 63  7  3 40  7 63  7
  7 63 10 40  7  7 63  7 63 63  3 63 63  7  7 63  3  7 63 63 63  3 10  3
 63 63  7  7 63 40  7 63  7 40  7 21 40 63  3 10  3 63 40 63 63  7  3  3
 21  7 63  3 63 21 40 40 63 40  7 63 63 40  5 10  7 63  3 10  3 10 63 10
 63 63], α=0.17213
iter 1: dis(P)=1516.229, dis(Proj P)=1662.0, f(P)=30143.38, proj P=[ 7 63 63 63 40 63 63 21  7 63 63  3 10 63 63  7  7 63  7  3 40  7 63  7
  7 63 10 40  7  7 63  7 63 63  3 63 63  7  7 63  3  7 63 63 63  3 10  3
 63 63  7  7 63 40  7 63  7 40  7 21 40 63  3 10  3 63 40 63 63  7  3  3
 21  7 63  3 63 21 40 40 63 40  7 63 63 40  5 10  7 63  3 10  3 10 63 10
 63 63], Q=[ 6  6  6 33 84  6  6 66  9 51 66 62 82  6  6 56 

In [10]:
dis_(project_P(P_star))

1527.0